# Chapter 5 Markov Decision Process Reading Note

- discrete time, infinite horizon, dynamic program

1. Definitions and Properties:
    - Model specification
    - Examples: Renewal, optimal inventory, cake eating, optimal stopping
    - Optimality theory
    - Algorithm: VFI,HPI,HPI VS NI, OPI
2. Detailed applications
    - Optimal Inventories
    - Optimal saving and labor income: MDP representation, Implementation, timing, output
    - Optimal Investment: description, MDP representation, Implementation
    
3. Modified Bellman equation:
    - Structure estimation: definition, illustration, expected value function, optimality vs EV
    - Gumbel Max Trick
    - Optimal savings with stochastic return on wealth
    - Q-Factors
    - Operator factorization: Refactoring the Bellman operator, refactorization and optimality, refactored OPI

## Theorems


### Proposition 5.1.1. Optimality of MDP

If $\mathcal{M}=(\Gamma, \beta, r, P)$ is a MDP with the value function $v^*$ and Bellman operator $T$, then

1. $v^*$ is the unique solution to the Bellman equation
2. $\lim_{k\to\infty} T^k v=v^*$ for all $v\in\mathbb{R}^X$ (Global stability)
3. Bellman's principle of optimality holds,
4. at least one optimal policy exists.

### Lemma 5.1.2. $\beta P_\sigma$ is subgradient of $T$ at v, $\sigma$ is v-greedy

If $v\in\mathbb{R}^X$ and $\sigma\in\Sigma$ is $v$-greedy, the $\beta P_\sigma$ is a subgradient of $T$ at $v$.

## MDP model

Now actions have affects on rewards, next state, also transition probabilities. Compared to Optimal stopping (binary choice). Now the feasible actions depends on state and are not restricted to binary choices.

A controller who interacts with a state process $(X_t)_{t\ge 0}$ by choosing an action path $(A_t)_{t\ge 0}$ to maximize expected discounted rewards,

$$
\mathbb{E}\sum_{t\ge 0}\beta^t r(X_t,A_t)
$$

taking the initial state $X_0$ as given.

**Controller is clairvoyant: cannot choose actions that depends on future states**.

We take 

- $X$: finite state space
- $A$: finite action space
- $\Gamma: X\mapsto \mathscr{P}(A)$: a non-emptycorrespondence from $X$ to the subsets of $A$, i.e., $\Gamma(x)\neq \emptyset\,\,\forall x\in X$

**We define the Markov Decision Process as a 4-Tuple $\mathcal{M}=(\Gamma, \beta, r, P)$**

1. **Feasible correpondence** $\Gamma: X\mapsto \mathscr{P}(A)$, induces the **feasible state-action pairs**

$$
G:=\{(x,a)\in X\times A: x\in X, a\in \Gamma(x)\}
$$

2. **Discount factor**: $\beta\in(0,1)$
3. **Reward function**: $r\in\mathbb{R}^G$, $r:G\mapsto \mathbb{R}$
4. **Stochastic kernel**: $P:G\times X \mapsto \mathbb{R}_+$ satisfying:

$$
\sum_{x'\in X}P(x,a,x')=1,\,\,\forall (x,a)\in G
$$

**Bellman Equation** corresponding to $\mathcal{M}$:

$$
v(x) = \max_{a\in \Gamma(x)}\left\{r(x,a)+\beta\sum_{x'}v(x')P(x,a,x')\right\} \,\,\forall x\in X
$$



We can understand the Bellman equation as reducing an infinite-horizon problem to a two period problem involving the present and the future. 

Current actions influence:

1. Current reward
2. Expected discounted value from future states

In every ccase, **there is a tradeoff between maximizing current rewards and shifting probability mass towards states with high future rewards**.

# Optimality

## Policies and Lifetime values

Let $\mathcal{M} = (\Gamma, \beta,r,P)$ be a MDP.

**Feasible policy set**

$$
\Sigma:=\{\sigma\in A^X: \sigma(x)\in \Gamma(x)\,\,\forall x\in X\}
$$

**If we select a policy $\sigma\in\Sigma$, it is understood that we respond to the state $X_t$ with action $A_t =\sigma(X_t)$ for all $t$**.

Hence, the state evolving by drawing $X_{t+1}$ from the stochastic kernel under $\sigma$ becomes,

$$
P(X_t,\sigma(X_t),\cdot)
$$

We denote $(X_t)_{t\ge 0}$ as a $P_\sigma$-Markov when,

$$
P_\sigma(x,x') = P(x,\sigma(x),x')
$$

(Left hand side is $P_\sigma$-Markov, RHS is a stochastic kernel)

**Note $P_\sigma\in\mathscr{M}(\mathbb{R}^X)$. Hence, fixing a policy closes the loop in the state transition process and defines a Markov chain for the state.**

Under the policy $\sigma$, rewards at state $x$ are $r(x,\sigma(x))$.

We denote $r_\sigma(x) = r(x,\sigma(x))$ and $\mathbb{E}_x = \mathbb{E}[\cdot|X_0=x]$.

Then the **lifetime value of following $\sigma$ from initial state $x$ can be written as**

$$
v_\sigma(x) =\mathbb{E}_x \sum_{t\ge 0}\beta^t r_\sigma(X_t), \,\,\,\text{$(X_t)$ is a $P_\sigma$ Markov with $X_0=x$}
$$

If $\beta<1$ we have,

**$\sigma$-value function**
$$
v_\sigma =\sum_{t\ge 0}\beta^t P_\sigma^t r_\sigma = (I-\beta P_\sigma)^{-1} r_\sigma
$$


**Policy Operator Correspond to $\sigma$**

(since we already specify $\sigma$, no need for max)

$$
(T_\sigma v)(x) = r_\sigma(x)+\beta\sum_{x'\in X} v(x')P_\sigma(x,x')
$$

In vector form, we have,

$$
T_\sigma v = r_\sigma + \beta P_\sigma v
$$

Computationally, we can iterating with $T_\sigma$ can obtain an approximation of $v_\sigma$.

- For small state spaces, we can use $v_\sigma = (I-\beta P_\sigma)^{-1} r_\sigma$
- If $X$ is very large, inverting the matrix $(I-\beta P_\sigma)$ is computationally demanding. 
- In such setting, iterating $T_\sigma$ is better.

**Interpretation of Policy operator**

$(T^k_\sigma v)(x)$ is the payoff from following $\sigma$ and starting in state $x$ when lifetime is truncated to the finite horizon $k$ and $v$ provides a terminal payoff in each state.

## Defining Optimality

Given MDP $\mathcal{M} = (\Gamma,\beta, r, P)$ with $\sigma$-value function $\{v_\sigma\}_{\sigma\in\Sigma}$.

The **value function** corresponding to $\mathcal{M}$ is defined as 

$$
v^* = \bigvee_{\sigma\in\Sigma} v_\sigma, \,\,\,v^*(x) = \max_{\sigma\in\Sigma} v_\sigma(x)
$$

Interpretation: the maximal lifetime value we can extract from feasible behaviors. The maximum exist at each $x$ because $X, A$, hence $\Sigma$ is finite.

A policy $\sigma\in\Sigma$ is called **optimal policy** for $\mathcal{M}$ if $v_\sigma =v^*$. In other words, a policy is optimal if its lifetime value is maximal at each state.

**v-greedy**

Given $v\in\mathbb{R}^X$, we define a policy $\sigma\in\Sigma$ to be **v-greedy** if

$$
\sigma(x)\in\arg\max_{a\in\Gamma(x)} \left\{r(x,a)+\beta\sum_{x'} v(x')P(x,a,x')\right\}
$$

**In essence, a $v$-greedy policy treats $v$ as the correct value function and sets all actions accordingly.**

**Bellman's principle of optimality**

Bellman's principle of optimality is said to hold for the MDP $\mathcal{M}$ if

$$
\sigma\in\Sigma \,\,\text{is optimal for $\mathcal{M}\iff$$\sigma$ is $v^*$-greedy}
$$

**Bellman operator**

The Bellman operator corresponding to $\mathcal{M}$ is the self-map $T$ on $\mathbb{R}^X$ defined by 

$$
(Tv)(x)=\max_{a\in\Gamma(x)}\left\{r(x,a)+ \beta\sum_{x'}v(x')P(x,a,x')\right\}
$$

**The Bellman operator is the pointwise maximum of $\{T_\sigma\}_{\sigma\in\Sigma}$**, hence, we can express the Bellman operator as

$$
T = \bigvee_{\sigma\in\Sigma} T_\sigma
$$

## Optimality Theory

It is important to understand the importance of Bellman's principle of optimality. 

Greedy policies are relatively easy to compute.

Solving 

$$
\sigma(x) =\arg\max_{a\in\Gamma(x)} \left\{r(x,a)+\beta\sum_{x'\in X} v(x')P(x,a,x')\right\}
$$

at each $x$ is easier than trying to directly solve the problem of maximizing lifetime value, i.e,

$$
v^*(x) = \max_{\sigma\in\Sigma} v_\sigma(x)
$$

This is because $|\Sigma|\gg |\Gamma(x)|$. 

The Bellman's principle of optimality tells us that solving the overall problem reduces to computing a $v$-greedy policy with the right choice of $v$.

For optimal stopping problems, that choice is the value function $v^*$.

Intuitively, $v^*$ assigns a correct value to each state, in the sense of maximal lifetime value the controller can extract, so using $v^*$ to calculate greedy policies leads to the optimal outcome.

## Examples of MDP

### Renewal Problem: Rust(1987), engine replacement problem for bus

In each period, the superintendent decides whether or not to replace the engine of a given bus. Replacement is costly but delaying risks unexpected failure. 

Consider an abstract version of Rust's problem with **binary action** $A_t=\{0,1\}$, 

- when $A_t=1$, the state resets to some fixed **renewal state** $\bar x \in X$.

- when $A_t=0$, the state updates according to $Q\in \mathscr{M}(\mathbb{R}^X)$.

Given current state $x$ and action $a$, current reward $r(x,a)$ is received. The discount rate is $\beta$.

**Bellman equation**

$$
v(x)= \max\{r(x,1)+ \beta v(\bar x), r(x,0)+\beta\sum_{x'}v(x')Q(x,x')\}
$$

**MDP representation**

To set the problem up as an MDP we set 

- $A=\{0,1\}$
- $\Gamma(x) = A$ for all $x\in A$

We define the stochastic kernel as

$$
P(x,a,x') = a\mathbb{1}\{x'=\bar x\}+(1-a)Q(x,x')\tag{$(x,a)\in G, x'\in X$}
$$

Inserting this $P$, we get the MDP form of Bellman equation:

$$
v(x)=\max_{a\in\{0,1\}}\left\{r(x,a)+ a\beta v(\bar x) + (1-a)\beta\sum_{x'\in X}v(x')Q(x,x')\right\}
$$

### Optimal inventory management

A firm where a manager maximizes shareholder value. To simplify, we ignore exit option, hence the value of the firm is the EPV of future profits.

Assume the firm only sell one product with profit function $\pi_t$, and let $r>0$ be the interest rate.

**value of the firm** is

$$
V_0 = \mathbb{E}\sum_{t\ge 0} \beta^t \pi_t
$$

The firm faces **exogenous demand process**

$$
(D_t)\sim_{IID}\varphi\in\mathcal{D}(\mathbb{Z}_+)
$$

The **Inventory** $(X_t)_{t\ge 0}$ obeys the law of motion:

$$
X_{t+1} = f(X_t,D_t,A_t) = \max\{X_t-D_{t+1}, 0\}+A_t
$$

The **Action $A_t$ is the unit of stock ordered this period, which take one period to arrive.**


We assume that firms cannot sell more stock than they have at hand and can store at maximum $K$ items. We set the price equals to 1. 

**Profit function**

$$
\pi_{t} = X_t\wedge D_{t+1}-cA_t - \kappa\mathbb{1}\{A_t>0\}
$$

- $c$ is the per unit cost, $\kappa$ is the fixed cost related to order inventory
- $X_t\wedge D_{t+1}$, implies firm cannot sell more than they have at hand.

### MDP formulation

Let $X$ be the **state space** of inventory at storage, i.e.,

$$
X = \{0,1,2,\cdots,K\}
$$

Then the **feasible correspondence** is,

$$
\Gamma(x) = \{0,1,2,\cdots,K-x\}
$$

**Reward function is the current expected profit**

$$
r(x,a) = \left(\sum_{d\ge 0} (x\wedge d)\varphi(d)\right)-ca-\kappa\mathbb{1}\{a>0\}
$$

**Stochastic kernel**

$$
P(x,a,x') = \mathbb{P}\{f(x,D,a)=x'\}, \tag{$D\sim \varphi$}
$$

**Bellman Equation**

$$
v(x)= \max_{a\in\Gamma(x)}\left\{r(x,a)+\beta\sum_{x'\in X} v(x')P(x,a,x')\right\}
$$

or

$$
v(x) =\max_{a\in\Gamma(x)}\left\{r(x,a)+\beta\sum_{d\ge 0} v(f(x,a,d))\varphi(d)\right\}
$$

**Bellman operator**

$$
(Tv)(x) = \max_{a\in \Gamma(x)}\left\{r(x,a)+\beta\sum_{d\ge 0}v(f(x,a,d))\varphi(d)\right\}
$$

The operator maps $\mathbb{R}^X$ to itself is designed so that its set of fixed points coincide with the solution of the Bellman equation.

### Cake Eating

A household with finite wealth endowment but has no labor income. The wealth evolves as

$$
W_{t+1} = R(W_t-C_t)
$$

where $C_t$ is current consumption, $R$ is the gross interest rate. Household has utilty function $u(C_t)$ and aim to maximize lifetime utility

$$
\mathbb{E}\sum_{t\ge 0}\beta^t u(C_t), W_0=w
$$

by choosing the consumption path or similarly by choosing the next period wealth level $w'=R(w-c)\implies c = w-w'/R$

**Bellman equation**

$$
v(w) = \max_{0\le w'\le w} \{u(w-w'/R)+\beta v(w')\}
$$

Hence, household uses the Bellman equation to trade-off current utility with future value.

**MDP formulation**

A MDP is a 4-tuple $\mathcal{M}=(\Gamma,\beta, r, P)$.

With $W$ the wealth space as the state space. We can first formulate the feasible correspondence $\Gamma(w)$.

Since houshold can only consume less or equal to that what they have and also greater than 0, this implies, with $W_0=w$, we have

$$
\Gamma(w) = \{0,1,\cdots, w\},\tag{$w\in W$}
$$

Then, we have discount factor $\beta = 1/(1+r)$ where $r=R-1$. Hence, $\beta =1/R$.

The reward function is the utility function, we have,

$$
u(c)= u(w,w') = u(w-w'/R)
$$

Since there is no uncertainty in this case, there is no stochastic kernel. 

### Optimal Stopping 

We can frame optimal stopping problem as MDP by adding new state variable

Consider a job search model with Markov wage, i.e., $(W_t)_{t\ge 0}$ is a $Q$-Markov on finite $W$.

To express the job search problem as MDP, we let

$$
X = \{0,1\}\times W
$$

be a state space whose typical element is $(e,w)$ with $e$ representing the employment status.

The action space $A=\{0,1\}$ denoting rejecting or accepting the wage offer.

# Algorithm

## Value function iteration

Value function iteration for MDPs is similar to VFI for the job search model.

We use successive approximation on $T$ to compute an approximation $v_k$ to the value function $v^*$ and then take a $v_k$-greedy policy.

- Easy to understand and implement
- less efficient compared to HPI and OPI

## Howard Policy Iteration

Howard Policy Iteration computes optimal policies by iterating between computing the value of a given policy and computing the greedy policy associated with that value. 

1. Given initial $\sigma\in\Sigma$ as the inital policy
2. Obtain the $\sigma$-value function $v_\sigma$
3. Get $v_\sigma$-greedy policy, $\sigma'$     (Policy Improvement)
4. Get $\sigma'$-value function, $v_\sigma$'   (Policy Evaluation)

$\vdots$


ATTRACTIVE:

1. In finite state setting, the algorithm always converges to an exact optimal policy in a finite number of steps, regardless of the initial condition.

2. The rate of convergence is faster than VFI

### HPI as Newton Iteration


Since $T$ is not always differentiable, we can substitute derivatives using **subgradient**


Given a self-map $T:S\mapsto S$, $S\subset \mathbb{R}^n$. An $n\times n$ matrix $D$ is called **subgradient** of $T$ at $v\in S$ if

$$
Tu \ge Tv + D(u-v) \,\,\forall u\in S
$$

- If $T$ convex and differentiable, there is only 1 subgradient equals to the derivative

- If $T$ convex but non-differentiable, then there may exist multiple subgradient.

Lemma 5.1.2.: For $v\in\mathbb{R}^X$, $\sigma$ is v-greedy, then $\beta P_\sigma$ is the subgradient of $T$ at $v$.

We use this, from Newton's iteration using subdifferntial, we can find this leads us to iterate on

$$
v_{k+1} = Qv_k
$$

where,

$$
Qv = (I-\beta P_\sigma)^{-1} (Tv - \beta P_\sigma v)
$$

where $\sigma$ is $v$-greedy policy.

Since $\sigma$ is $v$-greedy, this implie $Tv=T_\sigma v=r_\sigma + \beta P_\sigma v$.

Hence, we have,

$$
Qv = (I-\beta P_\sigma)^{-1} r_\sigma
$$

this is the HPI policy evaluation to update the $\sigma$-value function. 

The fact that HPI is a version of Newton's method suggests that its iterates $(v_k)_{k\ge 0}$ enjoys quadratic convergence. 

Under mild conditions, one can show there exists a constant $N$ such that for all $k\ge 0$,

$$
\|v_{k+1}-v_k\|\le N\|v_k-v_{k-1}\|^2
$$

Hence, HPI enjoys both a fast convergence rate and the robustness of global convergence.

However, HPI is not always optimal in terms of efficiency, since the size of the constant term also matters. This term can be large because at each step, the update from $v_\sigma$ to $v_{\sigma'}$ **requires computing the exact lifetime value** $v_{\sigma'}$ of the $v_\sigma$-greedy policy $\sigma'$.

Computing this fixed point exactly can be computationally expensive in high dimensions.

## Optimistic Policy Iteration (OPI)

Optimistic Policy Iteration borrows from VFI and HPI.

The algorithm is the same as HPI but instead of computing the full value $v_\sigma$ of a given policy, the approximation $T^m_\sigma v$ is used.


In the algorithm, the policy operator $T_{\sigma_k}$ is applied $m$ times to generate an approximation of $v_{\sigma_k}$.

Then constant step size $m$ can also be replaced with a sequence $(m_k)\subset \mathbb{N}$. 

In either case, for MDPs, convergence to an optimal policy is guaranteed.

**Convex Combination of VFI and HPI**

As $m\to\infty$, the algorithm increasingly approximates Howard Policy Iteratio, since $T^m_{\sigma_k}$

At the same time, as $m=1$, OPI reduces to VFI. 

In almost all dynamic programming application, there exists choices of $m>1$ such that OPI converges faster than VFI.

# Application

## Optimal Inventories